In [51]:
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
import os
import json
import google.generativeai as genai


In [57]:

# Initialize Pinecone
pc = Pinecone(api_key="fdcace68-7ad0-4748-ab34-c1ba93781b0e")


pc.create_index(
    name="rag-new",
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)


In [59]:
data =  json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Biology',
  'stars': 4,
  'review': 'Engaging lectures and fair grading. Could improve on office hours availability.'},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Brilliant instructor! Makes complex topics easy to understand. Highly recommended.'},
 {'professor': 'Dr. Sarah Thompson',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Interesting course material, but lectures can be a bit disorganized.'},
 {'professor': 'Prof. Robert Garcia',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Challenging course, but Prof. Garcia is always willing to help. Great problem-solving skills.'},
 {'professor': 'Dr. Amanda Lee',
  'subject': 'English Literature',
  'stars': 5,
  'review': 'Passionate about the subject. Encourages critical thinking and class discussions.'},
 {'professor': 'Prof. David Wilson',
  'subject': 'Chemistry',
  'stars': 2,
  'review': 'Difficult to follow in c

In [66]:
# Initialize the client (assuming your API key is already set)
client = genai

for review in data["reviews"]:
  response = client.embed_content(
      model="models/text-embedding-004",  # Update with the correct prefix
      content=review['review']
  )
  embedding = response["embedding"]
  processed_data.append(
      {
          "values": embedding,
          "id": review["professor"],
          "metadata": {
              "review": review["review"],
              "subject": review["subject"],
              "stars": review["stars"],
          }
      }
  )

In [67]:
processed_data[0]

{'values': [0.05819297,
  0.023434231,
  -0.039470267,
  0.016967973,
  0.02528193,
  0.0063500884,
  0.026547007,
  0.010625438,
  0.0031304955,
  0.024252629,
  0.08141971,
  0.017601974,
  0.033841822,
  -0.031210389,
  0.010948753,
  0.0010870263,
  0.043517902,
  0.0112123275,
  -0.08674798,
  0.016617693,
  -0.0014285739,
  -0.03926043,
  -0.022996975,
  -0.081698336,
  0.01151655,
  -0.009150575,
  0.004686964,
  -0.03026875,
  0.0033405314,
  -0.014795687,
  -0.013014632,
  0.01448622,
  -0.04702756,
  -0.031491697,
  -0.032052267,
  0.06838962,
  0.0010379438,
  0.014967208,
  0.00012421464,
  -0.041611843,
  -0.060060028,
  -0.018927706,
  0.009421811,
  0.03149194,
  -0.07072988,
  -0.002721107,
  -0.010093067,
  0.08032273,
  -0.04197532,
  0.10476437,
  -0.0033520143,
  0.029497445,
  -0.072827466,
  0.041040298,
  -0.017663045,
  -0.032538705,
  -0.006992113,
  -0.017146438,
  0.06686311,
  -9.8721255e-05,
  -0.030066578,
  -0.032355558,
  -0.021971695,
  -0.082713366,
  

In [68]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="nsl"
)

{'upserted_count': 20}

In [69]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'nsl': {'vector_count': 20}},
 'total_vector_count': 20}